In [ ]:
#general packages
import pandas as pd
from itertools import product
from barcode_key_converter import *

# This notebook is used to convert the old barcode key.

In [ ]:
#read in codebook
codebook = pd.read_csv("../../../barcode_key/codebook.csv", index_col = 4)

In [ ]:
codebook

In [ ]:
#keep only normal genes (old codebook generation may have missing fake codes)
blanks = codebook[codebook.index.str.startswith("blank")].index
codebook = codebook.drop(blanks)

In [ ]:
#generate all possible codewords
#range of all pseudocolors
ps = np.arange(1,7,1)
#generate all possible combinations
number_of_sites = 3
perm_ps = list(product(ps,repeat=number_of_sites))

In [ ]:
#make sure the length is equal pseudocolor^number of sites
len(perm_ps) == 6**3

In [ ]:
#make list of codes that has been used
codes = codebook[["hyb1","hyb2","hyb3"]].values
code_set = [tuple(i) for i in codes]

In [ ]:
#find unused codewords
diff = list(set(perm_ps) - set(code_set))
len(diff)

In [ ]:
#make fakebook
fakecodes = pd.DataFrame(diff)
fakecodes.columns = ["hyb1","hyb2","hyb3"]

# This cell block is if you need to add parity codes. Also double check how your parity code is generated and see if the last operation is + or -.

In [ ]:
#get fake code with error correction 
errorfakecode = []
total_pseudocolors = 6
for code in diff:
    error = (code[0]+code[1]+code[2]) % total_pseudocolors
    if error == 0:
        error = total_pseudocolors
    errorfakecode.append(int(error))
    
fakecodes["hyb4"] = errorfakecode

--------------------------------------------------------------------------------------------------------------------

In [ ]:
#make fake code names
fakegenes = []
for i in range(len(fakecodes)):
    fakegenes.append(f"fake{i}")

In [ ]:
#add fake names
fakecodes.index = fakegenes

In [ ]:
#add to codebook
codebook = pd.concat([codebook,fakecodes])

In [ ]:
codebook

In [ ]:
#is this encoded across channels?
across = True
#how many channels have signal
num_encoded_channels = 3

converter = barcode_key_converter(codebook, num_encoded_channels=num_encoded_channels)

if across == False:
    new_codebook = converter.within()
else:
    new_codebook = converter.across()

In [ ]:
#take a look to see it makes sense
new_codebook

In [ ]:
#write out new codebook
new_codebook.to_csv("../../../barcode_key/codebook_string_across.csv")